使用 GPT-4识别线上图像URL

In [1]:
import base64

from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model = "gpt-4-turbo",
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "介绍下这幅图?"},
                {
                    "type": "image_url",
                    "image_url":{
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    }
                }
            ]
        }
    ],
    max_tokens = 300,
)

print(response)

ChatCompletion(id='chatcmpl-9jMh2aSQ4B2xElFnwu1dBKlBrRtXI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图片展示了一条蜿蜒于郁郁葱葱的绿色草地和灌木丛中的木质步道。天空晴朗，蓝天中散布着几朵白云，给这个场景增添了一种安静和平和的氛围。阳光照亮了步道和周围的植被，使整个景色看起来生机勃勃。这种自然的设置可能是一个公园或自然保护区的一部分，是人们散步或亲近自然的理想地点。整个图像给人一种宁静和远离城市喧嚣的感觉。', role='assistant', function_call=None, tool_calls=None))], created=1720599020, model='gpt-4-turbo-2024-04-09', object='chat.completion', system_fingerprint='fp_7d35882d38', usage=CompletionUsage(completion_tokens=204, prompt_tokens=1121, total_tokens=1325))


In [2]:
print(response.choices[0].message.content)

这幅图片展示了一条蜿蜒于郁郁葱葱的绿色草地和灌木丛中的木质步道。天空晴朗，蓝天中散布着几朵白云，给这个场景增添了一种安静和平和的氛围。阳光照亮了步道和周围的植被，使整个景色看起来生机勃勃。这种自然的设置可能是一个公园或自然保护区的一部分，是人们散步或亲近自然的理想地点。整个图像给人一种宁静和远离城市喧嚣的感觉。


封装成一个函数 query image description

In [5]:
def query_image_description(url, prompt="介绍下这幅图？"):
    client = OpenAI()
    
    response = client.chat.completions.create(
        model = "gpt-4-turbo",
        messages = [
            {
                "role": "user",
                "content":[
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url":{"url":url}},
                ]
            }
        ],
        max_tokens = 1000,
    )
    
    return response.choices[0].message.content

In [6]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一个搞笑的对比图，左边展示了一只带有巨大肌肉的柴犬，被赋予“16岁的我”和描述符“我刚修完一片地图，身体发育旺盛，人见人爱的大暖男。”右边展示的是一只普通体型的柴犬，被标记为“工作后的我”，描述为“好累好累，好想被触摸，身体素质不行，我就好累小哥哥，脱发严重，求安慰。”

这是一种非常常见的幽默表达，用夸张的方式来描述人们在青少年时期和成年后工作压力下的巨大变化。通过对比两种极端的柴犬形象，象征性地表达了很多人对成长和日常生活压力的感受。


In [9]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()

def query_base64_image_description(image_path, prompt = "解释下图里的内容？", max_tokens = 1000):
    # 实现base64 编码
    def encode_base64(path):
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    
    # 获取图像的Base64 编码字符串
    base64_image = encode_base64(image_path)
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }
    
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content":[
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url":{"url": f"data:image/jpeg;base64,{base64_image}"}},
                ]
            }
        ],
        "max_tokens": max_tokens,
    }
    
    # 发送Http 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [14]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图展示的是1980年到2020年期间美国、中国、日本和德国的国内生产总值（GDP）的比较。图中的x轴代表时间（年份），y轴表示GDP总量，单位是万亿美元（Trillion USD）。

从图中可以观察到：

- 美国的GDP线（蓝色）呈现稳定的增长趋势，从1980年的约3万亿美元增长到2020年的超过20万亿美元。
- 中国的GDP线（红色）开始时较低，但从1990年代开始显著上升，特别是2000年后增长迅猛，到2020年接近15万亿美元。
- 日本的GDP线（紫色）在1990年代初达到高峰后遇到停滞，之后略有回落，整体保持在5万亿美元左右。
- 德国的GDP（绿色）整体呈现缓慢增长的趋势，从1980年的不到1万亿美元增长到2020年的约4万亿美元。

总结来说，这张图反映了过去40年中这四个国家经济发展的大致轨迹，其中中国的增长最为显著。


In [16]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一本笔记本上的笔记，主题是关于人工智能和机器学习中的各种技术。笔记内容包括：

1. **概述了PET技术** - 这可能指的是Prompt Engineering Techniques的缩写，这是一种用来提高语言模型效果的技术。

2. **Prompt Tuning (FMT + Small Model)** - 这里描述了Prompt Tuning的概念，强调了其在小模型上的应用。FMT可能是指某种特定的技术或框架。此外，还提到了关于输入`X`经过Embedding转换后得到新的表达式，以及变换后的输出`Y = WX`。

3. **Prefix Tuning** - 描述了Prefix Tuning技术，这是一种通过向模型输入添加固定前缀来调整模型的行为的方法。`W' = [Wp; W]`显示了新的权重矩阵是通过前置条件和原始权重矩阵组合而成，其中`Y = W'X`是变换后的输出。

4. **LoRA（Low-Rank Adaptation）技术** - 提到了Low-Rank Adaptation如何通过修改权重矩阵`W`来适应模型，使用`ΔW = AB`来描述权重变化，其中`A`和`B`代表低秩矩阵。

5. **性能对比** - 笔记的最后提到了两种LoRA技术的运算资源需求：Llama和QLoRA，分别需要78GB和48GB的内存。

这些笔记表明作者正在研究或学习高级的机器学习技术，特别是在自然语言处理和模型优化方面。


In [17]:
from IPython.display import display, Markdown
display(Markdown(content))

这张图片展示的是一本笔记本上的笔记，主题是关于人工智能和机器学习中的各种技术。笔记内容包括：

1. **概述了PET技术** - 这可能指的是Prompt Engineering Techniques的缩写，这是一种用来提高语言模型效果的技术。

2. **Prompt Tuning (FMT + Small Model)** - 这里描述了Prompt Tuning的概念，强调了其在小模型上的应用。FMT可能是指某种特定的技术或框架。此外，还提到了关于输入`X`经过Embedding转换后得到新的表达式，以及变换后的输出`Y = WX`。

3. **Prefix Tuning** - 描述了Prefix Tuning技术，这是一种通过向模型输入添加固定前缀来调整模型的行为的方法。`W' = [Wp; W]`显示了新的权重矩阵是通过前置条件和原始权重矩阵组合而成，其中`Y = W'X`是变换后的输出。

4. **LoRA（Low-Rank Adaptation）技术** - 提到了Low-Rank Adaptation如何通过修改权重矩阵`W`来适应模型，使用`ΔW = AB`来描述权重变化，其中`A`和`B`代表低秩矩阵。

5. **性能对比** - 笔记的最后提到了两种LoRA技术的运算资源需求：Llama和QLoRA，分别需要78GB和48GB的内存。

这些笔记表明作者正在研究或学习高级的机器学习技术，特别是在自然语言处理和模型优化方面。